In [1]:
import numpy as np
import pandas as pd

#### Stocks

In [2]:
xlsm = pd.ExcelFile('hist_data.xlsm')
stocks = ['AAPL','MSFT','F','BAC']

file_dfs = []

for stock in stocks:
    file_df = pd.read_excel(xlsm, stock, index_col=0, parse_dates=True)
    if 'Adj Close' in file_df.columns:
        file_dfs.append(file_df[['Adj Close']].rename(columns={'Adj Close': stock}))

file_agg = pd.concat(file_dfs, axis = 1)
file_agg

,AAPL,MSFT,F,BAC
Date,,,,
2022-10-31,152.041122,229.443207,11.974380,34.748581
2022-11-01,149.373917,225.529037,12.001248,34.902847
2022-11-02,143.801514,217.552444,11.696740,34.796783
2022-11-03,137.703613,211.770157,11.875863,34.603954
2022-11-04,137.435455,218.827515,12.099767,35.471703
...,...,...,...,...
2023-10-24,172.991058,329.860504,11.215128,25.268324
2023-10-25,170.657135,339.979980,11.362824,25.347691
2023-10-26,166.458023,327.225861,11.175742,25.913177


#### SOFR Curve

In [3]:
sofr = pd.read_excel(xlsm, 'SofrCurve', index_col=0, parse_dates=True)
sofr = sofr.iloc[:,1:].T
sofr.index = pd.to_datetime(sofr.index).rename('Date')
sofr = sofr.dropna(axis=1)
sofr.head()

/var/folders/t9/qjktny4x6ll5mcwrn8bs70s40000gn/T/ipykernel_41021/2493871371.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sofr = pd.read_excel(xlsm, 'SofrCurve', index_col=0, parse_dates=True)


Tenor,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
Date,,,,,,,,,,,,,,,,,,,,,
2022-10-31,0.039191,0.038721,0.038670,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.032350,0.030552,0.028708
2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.046970,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.036010,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02,0.039948,0.039286,0.039100,0.040852,0.044884,0.046580,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03,0.040389,0.039585,0.039350,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.047500,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.037380,0.037152,0.036870,0.036534,0.034424,0.032558,0.030723,0.028933


## 1. Calculate the daily risk factor changes

The relative return and log return are more appropriate for positive risk factors (e.g. equity prices, FX spot etc..). The advantage of log return is that it is additive. For zero rate curve risk factors, we use absolute daily changes

### 1a) Daily relative change in stocks price 

data_df_1 = file_agg.pct_change().dropna()
data_df_1

In [4]:
data_df = np.log(file_agg/file_agg.shift(1)).dropna()
data_df

,AAPL,MSFT,F,BAC
Date,,,,
2022-11-01,-0.017698,-0.017207,0.002241,0.004430
2022-11-02,-0.038019,-0.036009,-0.025700,-0.003043
2022-11-03,-0.043330,-0.026938,0.015198,-0.005557
2022-11-04,-0.001949,0.032782,0.018678,0.024767
2022-11-07,0.003895,0.028850,0.013966,0.005962
...,...,...,...,...
2023-10-24,0.002540,0.003668,-0.007871,-0.003919
2023-10-25,-0.013583,0.030217,0.013083,0.003136
2023-10-26,-0.024913,-0.038236,-0.016601,0.022064


### 1b) Absolute daily change in SOFR zero rate curve

In [5]:
sofr_return = (sofr - sofr.shift(1)).dropna()
sofr_return

Tenor,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-01,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,0.000521,0.000439,0.000342,0.000296,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,0.000234,0.000474,0.000404,0.000254,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,0.000690,0.001098,0.001085,0.000957,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,-0.000394,-0.000497,-0.000448,-0.000322,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,0.000429,0.000536,0.000499,0.000413,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,0.000114,0.000371,0.000417,0.000343,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.000100,0.000150,0.000393,0.000596,0.000743,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,-0.000410,-0.000747,-0.000950,-0.001055,...,-0.001020,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877


In [6]:
data_new = pd.concat([data_df, sofr_return], axis = 1)
data_new

,AAPL,MSFT,F,BAC,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-01,-0.017698,-0.017207,0.002241,0.004430,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02,-0.038019,-0.036009,-0.025700,-0.003043,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03,-0.043330,-0.026938,0.015198,-0.005557,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,-0.001949,0.032782,0.018678,0.024767,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07,0.003895,0.028850,0.013966,0.005962,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.002540,0.003668,-0.007871,-0.003919,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.013583,0.030217,0.013083,0.003136,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.000100,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,-0.024913,-0.038236,-0.016601,0.022064,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,...,-0.001020,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877


### 1c) Identify any nan data

In [7]:
nan_dates = data_new[data_new.isna().any(axis=1)].index
nan_dates

DatetimeIndex(['2022-11-11', '2023-04-07', '2023-06-19', '2023-10-09'], dtype='datetime64[ns]', name='Date', freq=None)

In [8]:
data_new[data_new.isna().any(axis=1)]

,AAPL,MSFT,F,BAC,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-11,0.019085,0.016854,0.022316,0.007316,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-07,NaN,NaN,NaN,NaN,0.000596,-0.000189,-0.001362,-0.001413,0.000175,0.000699,...,0.000680,0.000667,0.000657,0.000647,0.000637,0.000626,0.000542,0.000435,0.000439,0.000472
2023-06-19,NaN,NaN,NaN,NaN,0.000035,0.000088,0.000109,0.000121,0.000143,0.000175,...,-0.000029,-0.000033,-0.000035,-0.000037,-0.000039,-0.000039,-0.000035,-0.000034,-0.000029,-0.000024
2023-10-09,0.008416,0.007792,0.005816,0.009164,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Do the linear interpolation to fill the data 

In [9]:
data_new_df = data_new.interpolate(method ='linear')
data_new_df

,AAPL,MSFT,F,BAC,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-01,-0.017698,-0.017207,0.002241,0.004430,0.000413,0.000302,0.000216,0.000188,0.000272,0.000444,...,-0.000349,-0.000375,-0.000395,-0.000411,-0.000423,-0.000434,-0.000446,-0.000372,-0.000314,-0.000230
2022-11-02,-0.038019,-0.036009,-0.025700,-0.003043,0.000344,0.000262,0.000214,0.000128,0.000035,0.000132,...,0.000053,0.000019,-0.000013,-0.000040,-0.000058,-0.000064,-0.000018,-0.000043,0.000054,0.000130
2022-11-03,-0.043330,-0.026938,0.015198,-0.005557,0.000440,0.000299,0.000250,0.000302,0.000397,0.000527,...,0.000366,0.000368,0.000388,0.000409,0.000420,0.000409,0.000184,0.000198,0.000115,0.000047
2022-11-04,-0.001949,0.032782,0.018678,0.024767,0.005576,0.002758,-0.000555,-0.000543,-0.000069,-0.000355,...,0.000466,0.000488,0.000494,0.000495,0.000498,0.000512,0.000613,0.000424,0.000315,0.000278
2022-11-07,0.003895,0.028850,0.013966,0.005962,-0.005741,-0.003085,0.000665,0.000813,0.000243,0.000437,...,0.000431,0.000463,0.000487,0.000504,0.000512,0.000514,0.000498,0.000577,0.000577,0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.002540,0.003668,-0.007871,-0.003919,0.000015,0.000007,0.000011,0.000019,0.000052,0.000083,...,-0.000161,-0.000174,-0.000183,-0.000189,-0.000196,-0.000203,-0.000255,-0.000298,-0.000283,-0.000242
2023-10-25,-0.013583,0.030217,0.013083,0.003136,-0.000124,-0.000027,0.000057,0.000066,0.000059,0.000100,...,0.001196,0.001212,0.001226,0.001237,0.001244,0.001246,0.001202,0.001119,0.001126,0.001147
2023-10-26,-0.024913,-0.038236,-0.016601,0.022064,0.000066,0.000032,-0.000041,-0.000076,-0.000182,-0.000304,...,-0.001020,-0.001024,-0.001027,-0.001028,-0.001025,-0.001015,-0.000895,-0.000769,-0.000804,-0.000877


## 2. Calculate joint distribution of the daily risk factor changes

### 2a) Parametric and Monte Carlo Var

1-day risk factor changes are assumed to be multivariate-normally distributed. One can simply estimate the means and covariance matrix of the multivariate normal distribution using the sample `means` and `covariance matrix` calculated from the historical daily changes.

#### Calculate the means

In [10]:
mean_risk = pd.DataFrame(data_new_df.mean().T, columns = ['Mean Return'])
mean_risk

,Mean Return
AAPL,0.000408
MSFT,0.001498
F,-0.000855
BAC,-0.001243
1D,0.000056
1M,0.000057
2M,0.000058
3M,0.000052
6M,0.000035
9M,0.000026


#### Calculate the variance-covariance matrix

In [11]:
cov_risk = data_new_df.cov()
cov_risk

,AAPL,MSFT,F,BAC,1D,1M,2M,3M,6M,9M,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
AAPL,2.482670e-04,1.791690e-04,1.310566e-04,7.367619e-05,4.910597e-06,1.591467e-06,-4.590581e-07,-3.733748e-07,1.564113e-08,-1.864894e-07,...,-1.421053e-06,-1.413006e-06,-1.397604e-06,-1.374995e-06,-1.345473e-06,-1.309123e-06,-1.143430e-06,-1.189344e-06,-1.157774e-06,-1.127074e-06
MSFT,1.791690e-04,3.137999e-04,1.321883e-04,6.866378e-05,4.919860e-06,1.745901e-06,-6.470670e-07,-7.024751e-07,-4.622727e-07,-8.154872e-07,...,-9.539776e-07,-9.289964e-07,-8.997913e-07,-8.654191e-07,-8.254702e-07,-7.793380e-07,-5.672977e-07,-5.788035e-07,-5.534200e-07,-5.384185e-07
F,1.310566e-04,1.321883e-04,5.155809e-04,1.690935e-04,5.203595e-06,2.272617e-06,2.552793e-07,3.558528e-07,1.299235e-06,1.598028e-06,...,-1.316482e-06,-1.343987e-06,-1.367229e-06,-1.381680e-06,-1.384615e-06,-1.373370e-06,-1.274611e-06,-1.439287e-06,-1.441226e-06,-1.389531e-06
BAC,7.367619e-05,6.866378e-05,1.690935e-04,2.682169e-04,4.080173e-06,1.865740e-06,5.137577e-07,1.119301e-06,3.085988e-06,4.006378e-06,...,9.070452e-07,8.426289e-07,7.888951e-07,7.454073e-07,7.112025e-07,6.857828e-07,5.792372e-07,3.786579e-07,4.442893e-07,5.193375e-07
1D,4.910597e-06,4.919860e-06,5.203595e-06,4.080173e-06,1.935126e-06,7.077145e-07,-1.187314e-07,-9.803333e-08,2.669587e-08,-4.624695e-08,...,-8.310360e-08,-8.119565e-08,-8.027188e-08,-7.947037e-08,-7.812335e-08,-7.568759e-08,-5.902895e-08,-6.847063e-08,-7.404170e-08,-7.922285e-08
1M,1.591467e-06,1.745901e-06,2.272617e-06,1.865740e-06,7.077145e-07,3.561458e-07,1.514501e-08,-1.041441e-08,3.943683e-08,2.436693e-08,...,-1.491814e-08,-1.422539e-08,-1.361967e-08,-1.296968e-08,-1.218772e-08,-1.119226e-08,-7.465674e-09,-1.385872e-08,-1.586301e-08,-1.749719e-08
2M,-4.590581e-07,-6.470670e-07,2.552793e-07,5.137577e-07,-1.187314e-07,1.514501e-08,1.102684e-07,8.931059e-08,8.496510e-08,1.142346e-07,...,5.177159e-08,5.064952e-08,4.983554e-08,4.912046e-08,4.832595e-08,4.731614e-08,4.110854e-08,3.959784e-08,3.656905e-08,3.376858e-08
3M,-3.733748e-07,-7.024751e-07,3.558528e-07,1.119301e-06,-9.803333e-08,-1.041441e-08,8.931059e-08,1.031211e-07,1.332033e-07,1.717226e-07,...,6.964595e-08,6.707393e-08,6.486437e-08,6.284837e-08,6.087701e-08,5.883628e-08,4.883549e-08,4.410182e-08,4.083124e-08,3.882131e-08
6M,1.564113e-08,-4.622727e-07,1.299235e-06,3.085988e-06,2.669587e-08,3.943683e-08,8.496510e-08,1.332033e-07,2.757106e-07,3.626435e-07,...,1.486499e-07,1.423061e-07,1.367011e-07,1.316119e-07,1.268299e-07,1.221965e-07,1.001928e-07,8.344234e-08,7.583312e-08,7.366263e-08
9M,-1.864894e-07,-8.154872e-07,1.598028e-06,4.006378e-06,-4.624695e-08,2.436693e-08,1.142346e-07,1.717226e-07,3.626435e-07,4.911163e-07,...,2.225188e-07,2.133485e-07,2.052872e-07,1.979782e-07,1.910951e-07,1.843874e-07,1.522511e-07,1.281814e-07,1.167332e-07,1.135241e-07


## 3. Calculate distribution of 1-day portfolio P&L

#### 3a) Finding the sensitivity of the risk factor

#### For stocks

In [12]:
# Sensitivity of stocks is the notional of the positions 
# The notional position for each stocks is 1 million
num_stocks = 4
weight_stocks = np.array([1e6]*4)

#### For swap payer 

In [13]:
sofr.head()

Tenor,1D,1M,2M,3M,6M,9M,1Y,2Y,3Y,4Y,...,15Y,16Y,17Y,18Y,19Y,20Y,25Y,30Y,35Y,40Y
Date,,,,,,,,,,,,,,,,,,,,,
2022-10-31,0.039191,0.038721,0.038670,0.040536,0.044577,0.046004,0.046449,0.044583,0.042002,0.040318,...,0.037151,0.037057,0.036907,0.036698,0.036433,0.036111,0.034091,0.032350,0.030552,0.028708
2022-11-01,0.039604,0.039023,0.038886,0.040725,0.044849,0.046448,0.046970,0.045022,0.042344,0.040614,...,0.036802,0.036682,0.036511,0.036287,0.036010,0.035678,0.033645,0.031979,0.030238,0.028478
2022-11-02,0.039948,0.039286,0.039100,0.040852,0.044884,0.046580,0.047203,0.045496,0.042749,0.040868,...,0.036855,0.036701,0.036498,0.036248,0.035953,0.035613,0.033627,0.031936,0.030292,0.028608
2022-11-03,0.040389,0.039585,0.039350,0.041154,0.045281,0.047107,0.047894,0.046594,0.043833,0.041825,...,0.037221,0.037069,0.036886,0.036657,0.036372,0.036022,0.033811,0.032134,0.030407,0.028655
2022-11-04,0.045965,0.042343,0.038795,0.040611,0.045212,0.046752,0.047500,0.046097,0.043385,0.041503,...,0.037687,0.037557,0.037380,0.037152,0.036870,0.036534,0.034424,0.032558,0.030723,0.028933


In [14]:
# Sensitivity of swap requiring the pricing of the swap
# Sensitivity is the partial of PV01 w.r.t to the corresponding zero rate
# This measures how the price of a swap changes when the corresponding yield curve shifts by one basis point (0.01%).

def calculate_swap_price(rates, notional, strike):
    # Relevant payment tenors for the swap pricing
    tenors = ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y','7Y', '8Y', '9Y', '10Y']
    swap_prices = {}

    for start_date in rates.index:
        try:
            rates_data = rates.loc[start_date, tenors]
            
        except KeyError:
            print(f"KeyError: No data for {start_date} in rates.")
            continue  
            
        # PV of fixed leg
        pv_fixed = 0
        fixed_cf = notional * strike
        for t, rate in enumerate(rates_data, start = 1):
            df_fixed = np.exp(-rate * t)
            pv_fixed += fixed_cf * df_fixed

        # PV of floating leg
        pv_float = 0
        zero_rate_start = rates.loc[start_date,'1D']  # for the first floating payment
        
        for t, rate in enumerate(rates_data, start = 1):
            zero_rate_end = rate
    
            # Calculate the discount factors
            df_start = np.exp(-zero_rate_start * (t - 1)) if t > 1 else 1
            df_end = np.exp(-zero_rate_end * (t))
            
            forward_rate = (df_start / df_end) - 1
            pv_float += (notional * forward_rate) * df_end

            zero_rate_start = zero_rate_end

        swap_prices[start_date] = pv_float - pv_fixed

    return pd.DataFrame(list(swap_prices.items()), columns=['Date', 'Swap Base Price']).set_index('Date')

In [15]:
swap_notional = 100e6
strike = 0.042
payment_freq = 1 # annual payment
swap_t = 10

sofr_price = calculate_swap_price(sofr, swap_notional, strike)
sofr_price

,Swap Base Price
Date,
2022-10-31,-2.958503e+06
2022-11-01,-3.044883e+06
2022-11-02,-2.954226e+06
2022-11-03,-2.487463e+06
2022-11-04,-2.367458e+06
...,...
2023-10-24,2.338622e+06
2023-10-25,3.202746e+06
2023-10-26,2.367237e+06


In [16]:
def calculate_swap_sensitivity(rates, notional, strike):
    # Relevant payment tenors for the swap pricing
    tenors = ['1D'] + ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y','7Y', '8Y', '9Y', '10Y']

    # Calculate the initial swap price
    original_price_df = calculate_swap_price(rates, notional, strike)

    # Calculate the sensitivity or partial PVO1
    sensitivity_swap_factor = {}
    
    for start_date in original_price_df.index:
        original_price = original_price_df.loc[start_date, 'Swap Base Price']
        sensitivity = {}
        
        for tenor in tenors:
            if tenor not in rates.columns:
                print(f"KeyError: No data for {tenor} in rates.")
                continue

            shifted_rates = rates.copy()
            shifted_rates.loc[start_date, tenor] += 0.0001 # shift the corresponding rate of tenor by 1bps

            # Calculate new swap price with the shifted corresponding zero rates for tenor (e.g., for 1Y tenor, only shift the zero rate curve for 1Y tenor only)
            new_price_df = calculate_swap_price(shifted_rates, notional, strike)

            if start_date not in new_price_df.index:
                print(f"Skipping {start_date}, no new price calculated.")
                continue

            new_price = new_price_df.loc[start_date, 'Swap Base Price']

            # Calculate the sensitivity prices 
            sensitivity[tenor] = (new_price - original_price) * 10000

        sensitivity_swap_factor[start_date] = sensitivity   

    return pd.DataFrame.from_dict(sensitivity_swap_factor, orient='index'), new_price_df

In [17]:
sofr_sensitivity, new_sofr_price = calculate_swap_sensitivity(sofr, swap_notional, strike)
sofr_sensitivity

,1D,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y
2022-10-31,0.0,4.009176e+06,7.682661e+06,1.110661e+07,1.429497e+07,1.724958e+07,1.999215e+07,2.252876e+07,2.485211e+07,2.696867e+07,7.168540e+08
2022-11-01,0.0,4.007088e+06,7.675919e+06,1.109522e+07,1.427803e+07,1.723017e+07,1.997729e+07,2.252425e+07,2.486065e+07,2.699228e+07,7.178853e+08
2022-11-02,0.0,4.006151e+06,7.668646e+06,1.108177e+07,1.426353e+07,1.721702e+07,1.996313e+07,2.250705e+07,2.484104e+07,2.697044e+07,7.172292e+08
2022-11-03,0.0,4.003387e+06,7.651825e+06,1.104577e+07,1.420904e+07,1.714553e+07,1.987221e+07,2.239644e+07,2.471735e+07,2.683856e+07,7.136853e+08
2022-11-04,0.0,4.004965e+06,7.659429e+06,1.106062e+07,1.422738e+07,1.716303e+07,1.988599e+07,2.240238e+07,2.470836e+07,2.680818e+07,7.123153e+08
...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,0.0,3.984976e+06,7.624255e+06,1.097424e+07,1.403813e+07,1.682297e+07,1.934301e+07,2.161577e+07,2.365554e+07,2.547448e+07,6.719205e+08
2023-10-25,0.0,3.984377e+06,7.618270e+06,1.095464e+07,1.399646e+07,1.674791e+07,1.922946e+07,2.146173e+07,2.345742e+07,2.522863e+07,6.645690e+08
2023-10-26,0.0,3.986011e+06,7.629665e+06,1.098589e+07,1.405566e+07,1.684146e+07,1.935961e+07,2.162793e+07,2.365958e+07,2.546731e+07,6.714353e+08
2023-10-27,0.0,3.986525e+06,7.634036e+06,1.099779e+07,1.407471e+07,1.686876e+07,1.938947e+07,2.165306e+07,2.367712e+07,2.547572e+07,6.713648e+08


In [18]:
new_sofr_price.head()

,Swap Base Price
Date,
2022-10-31,-2.958503e+06
2022-11-01,-3.044883e+06
2022-11-02,-2.954226e+06
2022-11-03,-2.487463e+06
2022-11-04,-2.367458e+06


#### 3b) Finding the mean portfolio 

In [19]:
# Non-matching tenors (like 1D, 1M, 2M, 3M, 6M, 9M, 11Y, 12Y, etc.) we assume those values will contribute zero to the swap pricing calculations.
# rationale: Since those tenors are not used in the discounting of the swap, their sensitivity values can effectively be treated as zero in your calculations.
matching_tenors = ['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y']
daily_swap_pnl = (sofr_sensitivity[matching_tenors].shift(1) * sofr_return[matching_tenors]).sum(axis=1)
daily_swap_pnl

2022-10-31         0.000000
2022-11-01    -86245.717013
2022-11-02     90659.431532
2022-11-03    467665.779698
2022-11-04    120084.023058
                  ...      
2023-10-24      4492.210118
2023-10-25    868192.610756
2023-10-26   -831096.073870
2023-10-27    -21509.005026
2023-10-30     97203.456789
Length: 251, dtype: float64

In [20]:
mean_stocks = np.dot(weight_stocks, mean_risk.loc[stocks])
mean_swap = daily_swap_pnl.mean()
mean_porto_l = mean_stocks + mean_swap
print('Mean Portfolio:', mean_porto_l.item()) ; print('Mean Stocks:', mean_stocks.item()) ; print('Mean Swap:', mean_swap)

Mean Portfolio: 23407.281029553666
Mean Stocks: -191.9149146246727
Mean Swap: 23599.195944178337


#### 3c) Finding the variance of portfolio

In [21]:
cov_stocks = cov_risk.loc[['AAPL', 'MSFT', 'F', 'BAC'], ['AAPL', 'MSFT', 'F', 'BAC']]
cov_stocks

,AAPL,MSFT,F,BAC
AAPL,0.000248,0.000179,0.000131,0.000074
MSFT,0.000179,0.000314,0.000132,0.000069
F,0.000131,0.000132,0.000516,0.000169
BAC,0.000074,0.000069,0.000169,0.000268


In [22]:
mean_sensitivity_swap = sofr_sensitivity.mean()
cov_swap = cov_risk.loc[mean_sensitivity_swap.index, mean_sensitivity_swap.index]
cov_swap

,1D,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y
1D,1.935126e-06,-6.285184e-08,-1.182200e-07,-1.278113e-07,-1.275606e-07,-1.240754e-07,-1.202426e-07,-1.161276e-07,-1.115227e-07,-1.070930e-07,-1.032663e-07
1Y,-6.285184e-08,6.394950e-07,6.879589e-07,6.096842e-07,5.538304e-07,5.011599e-07,4.579083e-07,4.257802e-07,4.014200e-07,3.809861e-07,3.614387e-07
2Y,-1.182200e-07,6.879589e-07,8.352052e-07,7.900052e-07,7.375368e-07,6.818689e-07,6.333399e-07,5.946474e-07,5.631797e-07,5.358120e-07,5.102299e-07
3Y,-1.278113e-07,6.096842e-07,7.900052e-07,7.792991e-07,7.430025e-07,6.988075e-07,6.580327e-07,6.235360e-07,5.940133e-07,5.675735e-07,5.429011e-07
4Y,-1.275606e-07,5.538304e-07,7.375368e-07,7.430025e-07,7.219365e-07,6.874429e-07,6.534999e-07,6.240633e-07,5.978756e-07,5.737646e-07,5.510575e-07
5Y,-1.240754e-07,5.011599e-07,6.818689e-07,6.988075e-07,6.874429e-07,6.619276e-07,6.348255e-07,6.101159e-07,5.873830e-07,5.660079e-07,5.456816e-07
6Y,-1.202426e-07,4.579083e-07,6.333399e-07,6.580327e-07,6.534999e-07,6.348255e-07,6.140877e-07,5.943410e-07,5.750591e-07,5.562051e-07,5.380443e-07
7Y,-1.161276e-07,4.257802e-07,5.946474e-07,6.235360e-07,6.240633e-07,6.101159e-07,5.943410e-07,5.790421e-07,5.630281e-07,5.465883e-07,5.304202e-07
8Y,-1.115227e-07,4.014200e-07,5.631797e-07,5.940133e-07,5.978756e-07,5.873830e-07,5.750591e-07,5.630281e-07,5.499395e-07,5.359937e-07,5.218185e-07
9Y,-1.070930e-07,3.809861e-07,5.358120e-07,5.675735e-07,5.737646e-07,5.660079e-07,5.562051e-07,5.465883e-07,5.359937e-07,5.243830e-07,5.120988e-07


In [23]:
var_porto_l = np.dot(np.dot(weight_stocks, cov_stocks), np.transpose(weight_stocks))+ np.dot(np.dot(mean_sensitivity_swap, cov_swap), np.transpose(mean_sensitivity_swap))
print('Portfolio variance:', var_porto_l)

Portfolio variance: 404563241789.93085


## Parametric VaR Model:

### Expected Returns and Covariance Matrix

In [24]:
mu = data_new_df[['AAPL', 'MSFT', 'F', 'BAC']].mean()
sigma = data_new_df[['AAPL', 'MSFT', 'F', 'BAC']].cov()

print("Expected Returns:")
print(mu)
print("")
print("============================================")
print("")
print("Covariance Matrix:")
print(sigma)

Expected Returns:
AAPL    0.000408
MSFT    0.001498
F      -0.000855
BAC    -0.001243
dtype: float64


Covariance Matrix:
          AAPL      MSFT         F       BAC
AAPL  0.000248  0.000179  0.000131  0.000074
MSFT  0.000179  0.000314  0.000132  0.000069
F     0.000131  0.000132  0.000516  0.000169
BAC   0.000074  0.000069  0.000169  0.000268


In [25]:
meanP = (np.inner(weight_stocks, mean_risk.loc[stocks].T) + daily_swap_pnl.mean())[0]
varianceP = np.dot(np.dot(weight_stocks, cov_stocks), weight_stocks.T
                  ) + np.dot(np.dot(mean_sensitivity_swap, cov_swap), mean_sensitivity_swap.T)

print("==========================================")
print(f"Porfolio Expected P&L (mean) VaR: ${meanP:,.0f}")
print(f"Portfolio Variance: ${varianceP:,.0f}")
print("==========================================")

Porfolio Expected P&L (mean) VaR: $23,407
Portfolio Variance: $404,563,241,790


In [27]:
import scipy.stats as stat
from scipy.stats import norm

var_1d = np.abs(stat.norm.ppf(0.05, loc=meanP, scale=np.sqrt(varianceP))) 

print("====================================")
print(f"Parametric VaR [1d, 95%]: ${var_1d:,.0f}")
print("====================================")

Parametric VaR [1d, 95%]: $1,022,807
